In [1]:
import scipy.io
import pandas as pd
import numpy as np
import re
from os.path import join

from matplotlib import cm

In [2]:
def prt_create_dummy(fn='dummy.prt'):
    """create dummy empty prt file"""
    dummytxt = '\nFileVersion:        1\n\nResolutionOfTime:   Volumes\n\nExperiment:         <placeholderexpname>\n\nBackgroundColor:    0 0 0\nTextColor:          255 255 255\nTimeCourseColor:    255 255 255\nTimeCourseThick:    3\nReferenceFuncColor: 0 0 80\nReferenceFuncThick: 3\n\nNrOfConditions:     <placeholdernrconditions>\n\n'
    with open(fn, 'w') as f:
        f.write(dummytxt)
    return   


def prt_add_conditions(df, dummy_fn, new_fn, stepsize=0):
    """add condition matrix to prt"""
    # load the dummy file
    f = open(dummy_fn,'r')
    dtxt = f.read()
    f.close

    # get new text
    newtxt = '{}{}'.format(dtxt, add_conditions(df, stepsize=stepsize))

    # save the file under new filename
    with open(new_fn, 'w') as f:
        f.write(newtxt)
    return
       
    
def prt_fill_placeholders(fn, expname, nrcond):
    """set parameters"""
    # load the file 
    f = open(fn,'r')
    dtxt = f.read()
    f.close()
    
    # substitude the placeholder names
    dtxt = re.sub('<placeholderexpname>', str(expname), dtxt)
    dtxt = re.sub('<placeholdernrconditions>', str(nrcond), dtxt)
    
    # resave the file
    with open(fn, 'w') as f:
        f.write(dtxt)
    return
 
    
def add_conditions(df, stepsize=0):
    """add conditions based on columns of an input dataframe put in text format
    return array of strings"""
    # create empty array
    condstr_s = ''
    colors = color_grad(len(df.columns))
    clr_idx = 0
    # loop over all the conditions
    for colmn in df.columns:
        # get boolean encoding and set to volume numbers
        indx_val = np.array(df[colmn].index[df[colmn] == 1].to_list()) + 1      # take volume/idx of df columns
        vol_array = con_from_to(indx_val, stepsize=stepsize)                                       # get start / end volume
        # if column is a float, set to string and get color gradiant
        if is_float(colmn): 
            colmn = '{:0.3f}'.format(colmn)
            clr = colors[clr_idx]
            clr_idx += 1
        else:
            clr = None
        condstr = set_cond_str(colmn, vol_array, setcolor=clr)                    # put values in string format
        condstr_s += condstr                                                    # add all strings
    return(condstr_s)


def consecutive(data, stepsize=1):
    """get consec values in an array"""
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)


def con_from_to(data):
    """get from to value"""
    data = consecutive(data)
    return(np.array([[np.min(i), np.max(i)] for i in data]))
    
    
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
    
def random_rgb():
    """get a random, bright rgb color"""
    color = np.random.randint(0,256, size=2)
    color = np.append(color, 255)
    np.random.shuffle(color)
    return(color)


def set_cond_str(cond_nm, repsarray, setcolor=None):
    """generate string for the prt file (actual condition str)"""
    # loop over repetitions and put them in the correct text format
    rep_string = '' # start with empty string
    for reps in repsarray:
        rep_string += '{: 4d} {: 4d}\n'.format(reps[0], reps[1])
    
    # take random color
    if (setcolor is not None): curcolor = setcolor
    else: curcolor = random_rgb()

    # make current condtions string
    cond_str = '{}\n{}\n{}Color: {} {} {}\n\n'.format(cond_nm, len(repsarray), 
                                                      rep_string, curcolor[0], curcolor[1], 
                                                      curcolor[2])
    return(cond_str)


def consecutive(data, stepsize=1):
    """get consec values in an array"""
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)


def con_from_to(data,stepsize=0):
    """get from to value"""
    data = consecutive(data,stepsize=stepsize) # stepsize 0 is per volume encoding
    return(np.array([[np.min(i), np.max(i)] for i in data]))
    
    
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    

def rgb(x):
    """float to rgb"""
    return(int(np.round(x * 255)))

def color_grad(nr_col):
    cm_subsection = np.linspace(0, 1, nr_col)
    colors = np.array([ [rgb(cm.jet(x)[0]), rgb(cm.jet(x)[1]), rgb(cm.jet(x)[2])] for x in cm_subsection ])
    return(colors) 

# one function to create prts for all runs of a participant

In [3]:
# laod
runs = [1, 2, 3, 4, 5]
sub = 5
fpath = f'/media/jorvhar/Data1/OneDrive_Mint/Documenten/Matlab and Python short scripts/predmem/prtcreation/pp{sub}/'

fn = lambda run:'sub{}_run{}_timings.mat'.format(sub,run)
settingsfn = lambda run: '_sub{}_run{}_settings.mat'.format(sub,run)

stepsize = 1     # 0:per tr coding, 1: per same block coding
prune_vols = 3   # number of volumes to prune at the start of a condition = if 0 dont prune anything
block_len = 5    # length of blocks (used for pruning)

prt_create_dummy(fn='dummy.prt')
for run in runs:
    # load actual file
    mat = scipy.io.loadmat(join(fpath, fn(run)))
    settings_mat = scipy.io.loadmat(join(fpath, settingsfn(run)))


    ## ONOFF HANDLING
    # get unique values 
    unique_vals = np.unique(mat['timings'][1,:])
    unique_vals = unique_vals[~np.isnan(unique_vals)]  # remove nans

    # on periods over all unique
    matching_indices = np.where( np.isin(mat['timings'][1,:], unique_vals))
    
    
    
    ## PRUNING START OF BLOCKS IF DESIRED
    # prep condition matrix
    cond_array = mat['timings'][1,matching_indices][0]
    
    # loop over indx per condition to prune
    for i in range(prune_vols):
        cond_array[i::block_len] = 0
    
    # import back into original array
    mat['timings'][1,matching_indices] = cond_array
    # save RANOn
    ranon_matches = np.where( mat['timings'][1,:] == 0)
    
    
    
    ## MAINFUNCTION HANDLING
    all_matches = {}

    # loop over all unique conditions
    for unique_value in unique_vals:

        # save all conditions
        all_matches[unique_value] = np.where( mat['timings'][1,:] == unique_value)
        
        
        
    ## ONEHOT ENCODING
    # set too bool array
    soundon_array = np.zeros(mat['timings'][1,:].shape)
    soundon_array[matching_indices] = 1

    # set to bool array per condition
    sound_arrayz = {}
    # encoding of start of condition
    sound_array = np.zeros(mat['timings'][1,:].shape)
    sound_array[ranon_matches] = 1
    sound_arrayz[0.0] = sound_array
    for key in all_matches.keys():
        sound_array = np.zeros(mat['timings'][1,:].shape)
        sound_array[all_matches[key]] = 1
        sound_arrayz[key] = sound_array
        
        
    ## DF CREATION
    # get condition naming
    cond_names = np.array([item for sublist in settings_mat['design']['conds'][0, 0] for item in sublist])
    # create dataframe for all the pulses
    onoff_df = pd.DataFrame({'SoundOn': soundon_array})
    # create dataframe for all the pulses
    df = pd.DataFrame(sound_arrayz)
    df.columns = ['RANon'] + cond_names.flatten().tolist() # add correct naming for conditions
    
    
    
    ## CONVERTION TO PRT
    # save design matrix of run
    prt_add_conditions(df, 'dummy.prt', 'r{}-conditions.prt'.format(run), stepsize=stepsize)
    prt_fill_placeholders('r{}-conditions.prt'.format(run), 'r{}-conditions'.format(run), len(df.columns))
    # save design matrix of run
    prt_add_conditions(onoff_df, 'dummy.prt', 'r{}-OnOff.prt'.format(run), stepsize=stepsize)
    prt_fill_placeholders('r{}-OnOff.prt'.format(run), 'r{}-OnOff'.format(run), len(onoff_df.columns))

# --NOT NEEDED--
if you want to go step by step you can run this

## Load the desired file

In [ ]:
runs = [1, 2, 3, 4, 5, 6, 7]
run = 1

mat = scipy.io.loadmat('/media/jorvhar/Data1/OneDrive_Mint/Documenten/Matlab and Python short scripts/predmem/prtcreation/Logs_S4/fMRI/Subject_4/sub4_run{}_timings.mat'.format(run))
settings_mat = scipy.io.loadmat('/media/jorvhar/Data1/OneDrive_Mint/Documenten/Matlab and Python short scripts/predmem/prtcreation/Logs_S4/fMRI/Subject_4/_sub4_run{}_settings.mat'.format(run))

In [ ]:
# get condition naming
cond_names = np.array([item for sublist in settings_mat['design']['conds'][0, 0] for item in sublist])

## get where which condition and on/offs


In [ ]:
# get unique values 
unique_vals = np.unique(mat['timings'][1,:])
unique_vals = unique_vals[~np.isnan(unique_vals)]  # remove nans

# on periods over all unique
matching_indices = np.where( np.isin(mat['timings'][1,:], unique_vals))

In [ ]:
all_matches = {}

# loop over all unique conditions
for unique_value in unique_vals:
    
    # save all conditions
    all_matches[unique_value] = np.where( mat['timings'][1,:] == unique_value)

## put everything into a onhot array

In [ ]:
# set too bool array
soundon_array = np.zeros(mat['timings'][1,:].shape)
soundon_array[matching_indices] = 1

# set to bool array per condition
sound_arrayz = {}
for key in all_matches.keys():
    sound_array = np.zeros(mat['timings'][1,:].shape)
    sound_array[all_matches[key]] = 1
    sound_arrayz[key] = sound_array

In [ ]:
# create dataframe for all the pulses
onoff_df = pd.DataFrame({'SoundOn': soundon_array})

In [ ]:
# create dataframe for all the pulses
df = pd.DataFrame(sound_arrayz)
df.columns = cond_names.flatten().tolist()

## convert to prt

In [ ]:
# save combined matrix
prt_create_dummy(fn='dummy.prt')

# save design matrix of run
prt_add_conditions(df, 'dummy.prt', 'r{}-conditions.prt'.format(run))
prt_fill_placeholders('r{}-conditions.prt'.format(run), 'r{}-conditions'.format(run), len(df.columns))


In [ ]:
# do same for on off

# save design matrix of run
prt_add_conditions(onoff_df, 'dummy.prt', 'r{}-OnOff.prt'.format(run))
prt_fill_placeholders('r{}-OnOff.prt'.format(run), 'r{}-OnOff'.format(run), len(onoff_df.columns))